In [1]:
from xs_lexer import Lexer
from xs_parser import Parser
import copy
# from xs_node import Node_letter, Node_condition

In [7]:
class Node_letter(object):

    def __init__(self, token, name):
        self.token = token
        self.name = name
        self.state = 0
        self.neg = 0
        self.childs_pos = []
        self.childs_neg = []
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_letter({})".format(self.token)


class Node_condition(object):

    def __init__(self, left, cond, right):
        self.left = left
        self.token = self.cond = cond
        self.right = right
        self.neg = 0
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_condition({}".format(self.token)

In [8]:
class Interpreter(object):

    def __init__(self):
        self.graph_list = {}
        self.all_histories = []
        pass

    def interpret(self, node, graph):
        if node:
            if type(node).__name__ == "Node_condition":
                return abs(self.apply_logical(node, node.left,
                           node.right, graph) - node.neg)
            node.visited = 1 
            if type(node).__name__ == "Node_letter":
                implies_node = graph.find_letter_in_implies(node.name)
                if implies_node:
                    for child in implies_node.childs_pos:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 1 
                    for child in implies_node.childs_neg:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 0 
                node.visited = 0 
                return abs(node.state - node.neg)

    def apply_logical(self, node, left, right, graph):
        if node.token.type == "AND":
            return self.interpret(left, graph) & self.interpret(right, graph)
        if node.token.type == "OR":
            return self.interpret(left, graph) | self.interpret(right, graph)
        if node.token.type == "XOR":
            return self.interpret(left, graph) ^ self.interpret(right, graph)

In [18]:
# rule1 = "A => (C ^ B) + E"
# rule2 = "F => D ^ (B ^ C)"
rule1 = "A => B ^ C " # A => B et A => C
# rule2 = "D => G ^ E"
rule3 = "C ^ E => F"

true_facts = "AD"

queries = "BCF"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
#     graph.add_new_AST(Parser(Lexer(rule2)).parse())
    graph.add_new_AST(Parser(Lexer(rule3)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    if graph.incoherent == 0:
        graph.resolve_simple()
    else:
        graph.resolve_complex()

    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

'Graph' object has no attribute 'resolve_complex'


In [17]:
for node in graph.implies_list:
    print(node)
print()
for node in graph.implies_list[0].childs_pos:
    print(hex(id(node)))

Node_letter(Token(LETTER, B))
Node_letter(Token(LETTER, C))
Node_letter(Token(LETTER, G))
Node_letter(Token(LETTER, E))
Node_letter(Token(LETTER, F))

0x10c7ea860


In [289]:
graph = Graph()
    
graph.add_new_AST(Parser(Lexer(rule1)).parse())
# graph.add_new_AST(Parser(Lexer(rule2)).parse())
graph.add_new_AST(Parser(Lexer(rule3)).parse())
    
graph.learn_facts(true_facts)
for node in graph.implies_list:
    graph.set_facts(node)
if graph.incoherent == 0:
    graph.resolve_simple()
else:
    graph.resolve_complex()    
for query in queries:
    graph.query(query)

Node_letter(Token(LETTER, A))
ici
Node_letter(Token(LETTER, A))
ici
error_rule_not_found 0x1120b0160 0x10c9694e0


KeyError: None

In [359]:
class Graph(object):

    def __init__(self):
        self.implies_list = []
        self.implies_name_list = []
        self.true_facts = ""
        self.incoherent = 0
        self.interpretor = Interpreter()
        self.full_history = {}
        self.list_copy = []

    def error(self, s):
        raise Exception(s)

    def add_new_AST(self, root):
        self.update_implies_list(root.right)
        self.update_graph(root.left, root.right)

    def update_implies_list(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.name not in self.implies_name_list:
                    self.implies_list.append(node)
                    self.implies_name_list.append(node.name)
            if type(node).__name__ == "Node_condition":
                self.update_implies_list(node.left)
                self.update_implies_list(node.right)

    def update_graph(self, left, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.neg == 1:
                    (self.find_letter_in_implies(node.name)
                        .childs_neg.append(left))
                else:
                    (self.find_letter_in_implies(node.name)
                        .childs_pos.append(left))
            if type(node).__name__ == "Node_condition":
                if node.token.type == "OR" or node.token.type == "XOR":
                    self.update_ind_dics(left, node)
                    self.incoherent += 1
                self.update_graph(left, node.left)
                self.update_graph(left, node.right)

    def update_ind_dics(self, left, node):
        if node.token.type == "OR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).or_dic[left] = node.right
            else:
                node.left.or_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).or_dic[left] = node.left
            else:
                node.right.or_dic[left] = node.left
        if node.token.type == "XOR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).xor_dic[left] = node.right
            else:
                node.left.xor_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).xor_dic[left] = node.left
            else:
                node.right.xor_dic[left] = node.left
            
    def find_letter_in_implies(self, name):
        for node in self.implies_list:
            if node.name == name:
                return node
        return None

    def learn_facts(self, true_facts):
        self.true_facts = true_facts

    def set_facts(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    self.set_facts(child)
                if node.token.value in self.true_facts:
                    node.state = 1 
            if type(node).__name__ == "Node_condition":
                self.set_facts(node.left)
                self.set_facts(node.right)
        
    def query(self, letter):
        if letter in self.full_history.keys():
            if self.full_history[letter] != None:
                print("{} is {}".format(letter, self.full_history[letter]))
                return self.full_history[letter]
        if letter in self.true_facts:
            print("{} is True".format(letter))
            return True
        else:
            print("{} is False".format(letter))
            return False

    def resolve_simple(self):
        for node in self.implies_list:
            self.full_history[node.name] = self.get_simple_final_state(node)
            if self.full_history[node.name] is None:
                if node.name in self.true_facts:
                    self.full_history[node.name] = True
                else:
                    self.full_history[node.name] = False
        return self.full_history
    
    def get_simple_final_state(self, node):
        history = []
        for child in node.childs_pos:
            res = self.interpretor.interpret(child, self)
            if res == 1:
                history.append(True)

        for child in node.childs_neg:
            res = self.interpretor.interpret(child, self)
            if res == 1:
                history.append(False)

        if len(set(history)) == 0:
            return
        elif len(set(history)) == 1:
            return history[0]
        else:
            self.error("Error contradiction found with letter {}"
                       .format(node.name))
   
    def merge_new_graph(self, new):
        if self.tmp_history == {}:
            self.tmp_history = new
            return
        for letter, state in new.items():
            if state != self.tmp_history[letter]:
                self.tmp_history[letter] = "IND"
                
    def resolve_complex(self):
        inc = 0
        self.check_ind_rules(self, self, None)
        self.tmp_history = {}
        for copy_graph in self.list_copy:
            hist = copy_graph.resolve_simple()
            print(hist)
            self.merge_new_graph(hist)
        self.full_history = self.tmp_history

    def check_ind_rules(self, origin_graph, current_graph, prev_rule):
        tot = 0
        
        for node in current_graph.implies_list:
            for rule, mirror in node.xor_dic.items():
                tot += 1
                if prev_rule:
                    node.childs_pos.append(prev_rule)
                    if type(mirror).__name__ == "Node_letter":
#                         print('ici_mirror', mirror.childs_pos, node.xor_dic)
                        mirror.childs_pos.append(prev_rule)
                    else:
                        self.add_recursively_rule(mirror, prev_rule)

                copy_graph_1 = copy.deepcopy(current_graph)
                copy_graph_2 = copy.deepcopy(current_graph)
                copy_rule = None
                
                if type(mirror).__name__ == "Node_condition":
                    copy_node = copy_graph_1.find_letter_in_implies(node.name)
                    copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph_1)
                    copy_node_2 = copy_graph_2.find_letter_in_implies(node.name)
                    copy_rule_2 = self.get_copy_rule(node, rule, copy_node_2, copy_graph_2)
                
                if prev_rule:
                    self.set_rules(rule, node, copy_graph_1, 3)
                    self.set_rules(rule, node, copy_graph_2, 4)
                    if copy_rule:
                        self.check_ind_rules(origin_graph, copy_graph_1, copy_rule)
                    else:
                        self.check_ind_rules(origin_graph, copy_graph_1, None)
                    if type(mirror).__name__ == "Node_condition":
                        self.check_ind_rules(origin_graph, copy_graph_2, copy_rule_2)
                    else:
                        self.check_ind_rules(origin_graph, copy_graph_2, None)
                else:
                    self.set_rules(rule, node, copy_graph_1, 1)
                    self.set_rules(rule, node, copy_graph_2, 2)
                    if copy_rule:
                        self.check_ind_rules(origin_graph, copy_graph_1, copy_rule)
                    else:
                        self.check_ind_rules(origin_graph, copy_graph_1, None)
                    self.check_ind_rules(origin_graph, copy_graph_2, None)
                    
            if tot != 0:
                break
        if tot == 0:
            origin_graph.list_copy.append(current_graph)
        
    def set_rules(self, rule, node, copy_graph, step):
        copy_node = copy_graph.find_letter_in_implies(node.name)
        copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph)      
        copy_mirror = copy_node.xor_dic[copy_rule]
        if step == 1:
            self.del_rule_in_childs(copy_mirror, copy_rule)
        if step == 2:
            self.del_rule_in_childs(copy_node, copy_rule)
        if step == 3:
            pass
        if step == 4:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            self.del_rule_in_childs(copy_node, copy_rule)
        
        copy_mirror.xor_dic.pop(copy_rule)
        copy_node.xor_dic.pop(copy_rule)
    
    def del_rule_in_childs(self, node, rule):
        if type(node).__name__ == "Node_letter":
            for index in range(0, len(node.childs_pos)):
                if node.childs_pos[index] == rule:
                    del node.childs_pos[index]
                    return
        else:
            self.delete_recursively_rule(node, rule)

    def delete_recursively_rule(self, node, rule):
        if node:
            if type(node).__name__ == "Node_letter":
                self.del_rule_in_childs(node, rule)
            if type(node).__name__ == "Node_condition":
                self.delete_recursively_rule(node.left, rule)
                self.delete_recursively_rule(node.right, rule)
                
    def add_recursively_rule(self, mirror, rule):
        if node:
            if type(node).__name__ == "Node_letter":
                node.childs_pos.append(rule)
            if type(node).__name__ == "Node_condition":
                self.add_recursively_rule(node.left, rule)
                self.add_recursively_rule(node.right, rule)
                
    def get_copy_rule(self, true_node, true_rule, copy_node, copy_graph):
        for index in range(0, len(true_node.childs_pos)):
            if true_node.childs_pos[index] == true_rule:
                return copy_node.childs_pos[index]
        print('error_rule_not_found', hex(id(true_rule)), hex(id(copy_node)), len(true_node.childs_pos))
        return 

In [360]:
print(len(graph.list_copy))

16


In [380]:
# rule1 = "A => (C ^ B) + E"
# rule2 = "F => D ^ (B ^ C)"
rule1 = "A => B ^ (C ^ D)"
# rule2 = "A => D ^ E"
rule3 = " B ^ (C ^ D) => F"

true_facts = "A"

queries = "BCF"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
#     graph.add_new_AST(Parser(Lexer(rule2)).parse())
    graph.add_new_AST(Parser(Lexer(rule3)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    
    if graph.incoherent == 0:
        graph.resolve_simple()
    else:
        graph.resolve_complex()
    
    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

{'B': True, 'C': True, 'D': True, 'F': True}
{'B': True, 'C': False, 'D': False, 'F': True}
{'B': False, 'C': True, 'D': False, 'F': True}
{'B': False, 'C': False, 'D': True, 'F': True}
B is IND
C is IND
F is True


In [358]:
for i in range(0, 10):
    if i == 5:
        continue
    print(i)

0
1
2
3
4
6
7
8
9


In [12]:
for node in graph.implies_list:
    print(node)
print()
for node in graph.implies_list[0].childs_pos:
    if node in graph.implies_list[0].xor_dic:
        print("ici", node)
    else:
        print("pas ici", node)
print()
# print(graph.implies_list[2].childs_pos)
print(graph.implies_list[0].xor_dic.keys())

Node_letter(Token(LETTER, B))
Node_letter(Token(LETTER, C))
Node_letter(Token(LETTER, G))
Node_letter(Token(LETTER, F))

ici Node_letter(Token(LETTER, A))

dict_keys([<xs_node.Node_letter object at 0x102fb7550>])


In [45]:
import copy

In [47]:
print(graph)
print(graph.implies_list)

t = copy.deepcopy(graph)
print(t)
print(t.implies_list)

[<xs_node.Node_letter object at 0x1041b4e10>, <xs_node.Node_letter object at 0x1041b49e8>, <xs_node.Node_letter object at 0x1041b4c18>]
[<xs_node.Node_letter object at 0x1041ce278>, <xs_node.Node_letter object at 0x1041b4240>, <xs_node.Node_letter object at 0x1041b42b0>]


In [21]:
t = copy.copy(graph.implies_list[0])

# for x in t:
print(x)
print()
for x in graph.implies_list[0].childs_pos:
    print(x)


Node_letter(Token(LETTER, F))

Node_letter(Token(LETTER, A))
Node_letter(Token(LETTER, F))


In [37]:
dic = {}
dic['a'] = "b"
dic['c'] = "d"

In [41]:
for i in dic.values():
    print(i)

b
d


In [35]:
dic.keys

<function dict.keys>

In [315]:
a = [1, 2]
t = {}
t[a[0]] = 10
t[a[1]] = 15
graph

In [316]:
print(t)
print(a)

{1: 10, 2: 15}
[1, 2]


In [317]:
for k, v in t.items():
    print(k, v)

1 10
2 15


In [318]:
t

{1: 10, 2: 15}

In [319]:
tt = None

In [321]:
if tt not in t:
    print('ici')

ici
